In [1]:
import time
import random
import numpy as np
import pandas as pd
from typing import TypeVar, Callable

from sklearn.preprocessing import MinMaxScaler
from sktime.classification import BaseClassifier
from sklearn.model_selection import StratifiedKFold
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import classification_report, accuracy_score
from sktime.classification.interval_based import TimeSeriesForestClassifier

In [2]:
raw_csv_data: pd.DataFrame = pd.read_csv('time_series.csv') 
df: pd.DataFrame = raw_csv_data.copy() 

In [3]:
df.head()

,id,timestamp,date,activity,owner,class,binary_class
0,0,2003-05-08 00:00:00,2003-05-08,0,condition_1,unipolar depressive,depressed
1,0,2003-05-08 00:01:00,2003-05-08,0,condition_1,unipolar depressive,depressed
2,0,2003-05-08 00:02:00,2003-05-08,0,condition_1,unipolar depressive,depressed
3,0,2003-05-08 00:03:00,2003-05-08,0,condition_1,unipolar depressive,depressed
4,0,2003-05-08 00:04:00,2003-05-08,0,condition_1,unipolar depressive,depressed


In [4]:
def to_2D_array(df: pd.DataFrame, n_days: int, variable: str) -> np.ndarray:
    n_users: int = len(df.id.unique())
    arr: np.ndarray = df[variable].values.reshape(n_users, n_days)
    return arr

In [5]:
X = to_2D_array(df, 1440, 'activity')
X.shape

(1029, 1440)

In [6]:
scaler = MinMaxScaler()

n_samples, n_timesteps = X.shape
X_reshaped = X.reshape(-1, 1)

X = scaler.fit_transform(X_reshaped)
X = X.reshape(n_samples, n_timesteps)

In [7]:
y = df.iloc[[ i*1440 for i in range(1029) ], -1].values
y.shape

(1029,)

In [8]:
def format_seconds(time_in_seconds: float) -> str:
    hours: int = int(time_in_seconds // 3600)
    minutes: int = int((time_in_seconds % 3600) // 60)
    seconds: int = int(time_in_seconds % 60)
    
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

In [9]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [10]:
Classifier = TypeVar('Classifier', bound=BaseClassifier)
def run_classifier(clf: Classifier, X_train: np.ndarray, X_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> np.ndarray:
    clf.fit(X_train, y_train) 
    y_pred: np.ndarray = clf.predict(X_test) 
    accuracy: float = accuracy_score(y_test, y_pred)
    print(f'Accuracy score: {accuracy}')
    print(classification_report(y_test, y_pred, digits=5, zero_division=np.nan))
    return accuracy

In [12]:
start_time: float = time.process_time()

accuracies_tsf = []
accuracies_rocket = []
for train_index, test_index in cv.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    tsf = TimeSeriesForestClassifier(random_state=42, n_jobs=-1)
    rocket = RocketClassifier(random_state=42, n_jobs=-1)
    accuracies_tsf.append(run_classifier(tsf, X_train, X_test, y_train, y_test))
    accuracies_rocket.append(run_classifier(rocket, X_train, X_test, y_train, y_test))
    print()

end_time: float = time.process_time()
cpu_execution_time: float = end_time - start_time
print(f'CPU Execution time: {format_seconds(cpu_execution_time)}')

ModuleNotFoundError: RocketClassifier requires package 'numba' to be present in the python environment, but 'numba' was not found. 'numba' is a soft dependency and not included in the base sktime installation. Please run: `pip install numba` to install the numba package. To install all soft dependencies, run: `pip install sktime[all_extras]`

In [ ]:
print(f'Acurácia Média TSF: {np.mean(accuracies_tsf):.2f}')    
print(f'Acurácia Média ROCKET: {np.mean(accuracies_rocket):.2f}')  

Acurácia Média TSF: 0.80
Acurácia Média LSTM: 0.65
